# Simple VPIN Benchmark

Basic VPIN performance testing with different data sizes.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import rust_indicators
from pycoingecko import CoinGeckoAPI
import time

cg = CoinGeckoAPI()
ta = rust_indicators.RustTA()
print(f"Backend: {ta.device()}")

In [ ]:
# Generate test data
def generate_test_data(size):
    np.random.seed(42)
    returns = np.random.normal(0.0001, 0.02, size)
    prices = 50000 * np.exp(np.cumsum(returns))
    
    volatility = np.abs(returns)
    volumes = 1e9 * (1 + 2 * volatility) * np.random.lognormal(0, 0.5, size)
    
    price_changes = np.diff(prices, prepend=prices[0])
    buy_ratio = 0.5 + 0.3 * np.tanh(price_changes / np.std(price_changes))
    
    buy_volumes = volumes * buy_ratio
    sell_volumes = volumes * (1 - buy_ratio)
    
    return buy_volumes, sell_volumes

print("Test data generator ready")

In [ ]:
# Benchmark different sizes
test_configs = [(1000, 10), (10000, 50), (50000, 200)]
results = []

for size, window in test_configs:
    print(f"\nTesting {size:,} points, window={window}")
    
    buy_vols, sell_vols = generate_test_data(size)
    
    times = []
    for i in range(5):
        start = time.time()
        vpin = ta.vpin(buy_vols, sell_vols, window)
        elapsed = time.time() - start
        times.append(elapsed)
    
    avg_time = np.mean(times)
    throughput = size / avg_time
    
    results.append({
        'size': size,
        'window': window,
        'time': avg_time,
        'throughput': throughput
    })
    
    print(f"  Time: {avg_time:.4f}s")
    print(f"  Throughput: {throughput:,.0f} points/sec")
    print(f"  VPIN range: {np.nanmin(vpin):.4f} - {np.nanmax(vpin):.4f}")

In [ ]:
# Plot results
df = pd.DataFrame(results)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['size'], y=df['throughput'],
    mode='lines+markers+text',
    text=[f"w={w}" for w in df['window']],
    textposition="top center",
    name='Throughput'
))

fig.update_layout(
    title='VPIN Performance vs Data Size',
    xaxis_title='Data Size',
    yaxis_title='Throughput (points/sec)',
    template='plotly_dark'
)

fig.show()

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Simple VPIN Benchmark\n",
    "\n",
    "Basic VPIN performance testing with different data sizes."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "import plotly.graph_objects as go\n",
    "import rust_indicators\n",
    "from pycoingecko import CoinGeckoAPI\n",
    "import time\n",
    "\n",
    "cg = CoinGeckoAPI()\n",
    "ta = rust_indicators.RustTA()\n",
    "print(f\"Backend: {ta.device()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate test data\n",
    "def generate_test_data(size):\n",
    "    np.random.seed(42)\n",
    "    returns = np.random.normal(0.0001, 0.02, size)\n",
    "    prices = 50000 * np.exp(np.cumsum(returns))\n",
    "    \n",
    "    volatility = np.abs(returns)\n",
    "    volumes = 1e9 * (1 + 2 * volatility) * np.random.lognormal(0, 0.5, size)\n",
    "    \n",
    "    price_changes = np.diff(prices, prepend=prices[0])\n",
    "    buy_ratio = 0.5 + 0.3 * np.tanh(price_changes / np.std(price_changes))\n",
    "    \n",
    "    buy_volumes = volumes * buy_ratio\n",
    "    sell_volumes = volumes * (1 - buy_ratio)\n",
    "    \n",
    "    return buy_volumes, sell_volumes\n",
    "\n",
    "print(\"Test data generator ready\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Benchmark different sizes\n",
    "test_configs = [(1000, 10), (10000, 50), (50000, 200)]\n",
    "results = []\n",
    "\n",
    "for size, window in test_configs:\n",
    "    print(f\"\\nTesting {size:,} points, window={window}\")\n",
    "    \n",
    "    buy_vols, sell_vols = generate_test_data(size)\n",
    "    \n",
    "    times = []\n",
    "    for i in range(5):\n",
    "        start = time.time()\n",
    "        vpin = ta.vpin(buy_vols, sell_vols, window)\n",
    "        elapsed = time.time() - start\n",
    "        times.append(elapsed)\n",
    "    \n",
    "    avg_time = np.mean(times)\n",
    "    throughput = size / avg_time\n",
    "    \n",
    "    results.append({\n",
    "        'size': size,\n",
    "        'window': window,\n",
    "        'time': avg_time,\n",
    "        'throughput': throughput\n",
    "    })\n",
    "    \n",
    "    print(f\"  Time: {avg_time:.4f}s\")\n",
    "    print(f\"  Throughput: {throughput:,.0f} points/sec\")\n",
    "    print(f\"  VPIN range: {np.nanmin(vpin):.4f} - {np.nanmax(vpin):.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot results\n",
    "df = pd.DataFrame(results)\n",
    "\n",
    "fig = go.Figure()\n",
    "fig.add_trace(go.Scatter(\n",
    "    x=df['size'], y=df['throughput'],\n",
    "    mode='lines+markers+text',\n",
    "    text=[f\"w={w}\" for w in df['window']],\n",
    "    textposition=\"top center\",\n",
    "    name='Throughput'\n",
    "))\n",
    "\n",
    "fig.update_layout(\n",
    "    title='VPIN Performance vs Data Size',\n",
    "    xaxis_title='Data Size',\n",
    "    yaxis_title='Throughput (points/sec)',\n",
    "    template='plotly_dark'\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Summary\n",
    "print(\"\\n\" + \"=\"*40)\n",
    "print(\"BENCHMARK SUMMARY\")\n",
    "print(\"=\"*40)\n",
    "for r in results:\n",
    "    print(f\"Size {r['size']:6,}: {r['throughput']:8,.0f} pts/s\")\n",
    "\n",
    "avg_throughput = np.mean([r['throughput'] for r in results])\n",
    "print(f\"\\nAverage: {avg_throughput:,.0f} points/sec\")\n",
    "print(\"=\"*40)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
